In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore')
import lightgbm as lgbm
from sklearn.multioutput import MultiOutputRegressor

newfeature = pd.read_csv('feature_CPAD.csv')
newfeature

CPAD = pd.read_csv('1540_CPAD.csv')
CPAD

,entry,length,uniprot_id,sequence,uni_length,pdb_id,mutation_type,temperature,ph,unit,value,yvalue,ndatapoints,intensity_avg
0,T-0001,192.0,P02763,MALSWVLTVLSLLPLLEAQIPLCANLVPVPITNATLDRITGKWFYI...,201,3KQ0,Wild Type,45,5.5,Minutes,"2.0,13.0,20.0,30.0,60.0,90.0,120.0,180.0,240.0...","1.68,3.85,7.46,43.67,58.86,83.9,93.49,106.41,1...",11,128.636364
1,T-0002,192.0,P02763,MALSWVLTVLSLLPLLEAQIPLCANLVPVPITNATLDRITGKWFYI...,201,3KQ0,Wild Type,45,5.5,Minutes,"18.0,30.0,60.0,90.0,120.0,180.0,240.0,300.0,360.0","10.11,38.74,115.01,152.07,162.42,176.61,196.1,...",9,155.333333
2,T-0003,192.0,P02763,MALSWVLTVLSLLPLLEAQIPLCANLVPVPITNATLDRITGKWFYI...,201,3KQ0,Wild Type,45,5.5,Minutes,"13.0,20.0,30.0,60.0,120.0,180.0,240.0,300.0,360.0","8.18,22.38,55.1,126.8,167.23,195.14,205.25,208...",9,147.000000
3,T-0004,192.0,P02763,MALSWVLTVLSLLPLLEAQIPLCANLVPVPITNATLDRITGKWFYI...,201,3KQ0,Wild Type,45,5.5,Minutes,"14.0,25.0,30.0,44.0,60.0,74.0,90.0,120.0,180.0...","26.23,40.9,73.15,105.87,131.86,145.57,157.36,1...",12,128.083333
4,T-0005,192.0,P02763,MALSWVLTVLSLLPLLEAQIPLCANLVPVPITNATLDRITGKWFYI...,201,3KQ0,Wild Type,60,5.5,Minutes,"10.0,14.0,20.0,30.0,60.0,90.0,120.0,180.0,240....","10.34,9.86,13.47,22.12,41.59,55.78,69.25,89.94...",11,129.454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,T-1747,184.0,Q5GN72,MALLWALAVLSLLPLLDAQSPECANLMTVAPITNATMDLLSGKWFY...,202,NaN,Wild Type,64,4.5,Hours,"0.45,0.89,1.29,1.59,1.86,2.22,2.58,3.02,3.46,3...","1.24,3.91,9.67,13.86,20.45,25.77,31.24,35.61,3...",51,10.998039
1536,T-1748,184.0,Q5GN72,MALLWALAVLSLLPLLDAQSPECANLMTVAPITNATMDLLSGKWFY...,202,NaN,Wild Type,64,5.5,Hours,"0.32,0.58,0.67,0.74,0.83,0.91,0.98,1.09,1.18,1...","6.22,8.58,14.76,21.09,28.97,40.06,48.62,56.46,...",77,9.935844
1537,T-1749,184.0,Q5GN72,MALLWALAVLSLLPLLDAQSPECANLMTVAPITNATMDLLSGKWFY...,202,NaN,Wild Type,67,5.5,Hours,"0.22,0.59,0.6,0.67,0.75,0.83,0.93,1.05,1.13,1....","0.49,14.61,3.88,26.47,37.0,47.8,58.12,61.84,70...",58,10.026897
1538,T-1750,184.0,Q5GN72,MALLWALAVLSLLPLLDAQSPECANLMTVAPITNATMDLLSGKWFY...,202,NaN,Wild Type,67,5.5,Hours,"0.16,0.3,0.42,0.56,0.67,0.76,0.83,0.92,1.0,1.0...","0.07,6.88,2.56,8.69,15.25,21.7,29.66,40.91,49....",83,9.664819


In [15]:
df = pd.concat([CPAD,newfeature],axis=1)
df

q3 = df.quantile(0.75)
q1 = df.quantile(0.25)

iqr = q3 - q1

def outlier(df):
    k = df['intensity_avg']
    if k > q3['intensity_avg'] + 1.5 * iqr['intensity_avg'] or k < q1['intensity_avg'] - 1.5 * iqr['intensity_avg']:
        return True
    else:
        return False

df['ountiler'] = df.apply(outlier, axis = 1)
df_trim = df.loc[df['ountiler'] == False]

del df_trim['ountiler']
df_trim

df_index = df_trim[df_trim['intensity_avg'].quantile(0.8) < df_trim['intensity_avg']].index
df_del = df_trim.drop(df_index)
df_del

,entry,length,uniprot_id,sequence,uni_length,pdb_id,mutation_type,temperature,ph,unit,...,PROTSCALE|Coil,PROTSCALE|Nomalized_frequency_alpha_helix,PROTSCALE|Nomalized_frequency_beta_sheet,PROTSCALE|Nomalized_frequency_beta_turn,PROTSCALE|Total_beta_strand,PROTSCALE|Antiparallel_beta_strand,PROTSCALE|Parallel_beta_strand,PROTSCALE|AA_composition,PROTSCALE|AA_composition_UniProtKB,PROTSCALE|Relative_mutability
6,T-0007,192.0,P02763,MALSWVLTVLSLLPLLEAQIPLCANLVPVPITNATLDRITGKWFYI...,201,3KQ0,Wild Type,60,5.5,Minutes,...,195.900,211.40,199.28,192.17,204.65,208.27,194.77,1145.0,1157.75,15410
7,T-0008,14.0,P22303,MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGG...,614,NaN,Single,37,7.0,Hours,...,616.354,610.23,608.58,619.15,611.90,614.00,610.11,3665.6,3675.16,44419
8,T-0009,14.0,P22303,MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGG...,614,NaN,Single,37,7.0,Hours,...,616.354,610.23,608.58,619.15,611.90,614.00,610.11,3665.6,3675.16,44419
9,T-0010,14.0,P22303,MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGG...,614,NaN,Single,37,7.0,Hours,...,616.354,610.23,608.58,619.15,611.90,614.00,610.11,3665.6,3675.16,44419
10,T-0011,14.0,P22303,MRPPQCLLHTPSLASPLLLLLLWLLGGGVGAEGREDAELLVTVRGG...,614,NaN,Single,37,7.0,Hours,...,616.354,610.23,608.58,619.15,611.90,614.00,610.11,3665.6,3675.16,44419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,T-1747,184.0,Q5GN72,MALLWALAVLSLLPLLDAQSPECANLMTVAPITNATMDLLSGKWFY...,202,NaN,Wild Type,64,4.5,Hours,...,193.904,217.18,198.20,189.59,203.00,204.39,200.01,1172.9,1189.21,15700
1536,T-1748,184.0,Q5GN72,MALLWALAVLSLLPLLDAQSPECANLMTVAPITNATMDLLSGKWFY...,202,NaN,Wild Type,64,5.5,Hours,...,193.904,217.18,198.20,189.59,203.00,204.39,200.01,1172.9,1189.21,15700
1537,T-1749,184.0,Q5GN72,MALLWALAVLSLLPLLDAQSPECANLMTVAPITNATMDLLSGKWFY...,202,NaN,Wild Type,67,5.5,Hours,...,193.904,217.18,198.20,189.59,203.00,204.39,200.01,1172.9,1189.21,15700
1538,T-1750,184.0,Q5GN72,MALLWALAVLSLLPLLDAQSPECANLMTVAPITNATMDLLSGKWFY...,202,NaN,Wild Type,67,5.5,Hours,...,193.904,217.18,198.20,189.59,203.00,204.39,200.01,1172.9,1189.21,15700


In [16]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures 
%matplotlib inline 
import matplotlib.pyplot as plt
from functools import reduce
import operator
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

train_rmse, val_rmse = [], []
train_r2, val_r2 = [], []
train_mae, val_mae = [], []
x10_label, y10_label = [], []

df_trainset = df_del
value = df_trainset['value']
yvalue = df_trainset['yvalue']

# Step 1: training data

for i in range(df_trainset.shape[0]):
    X_train = np.array([[float(v)] for v in value.iloc[i].split(',')])
    Y_train = np.array([[float(v)] for v in yvalue.iloc[i].split(',')])
    
# Step 2: data preparation
    n_degree = 5 
    poly_features = PolynomialFeatures(degree=n_degree, include_bias=True)
    x_train_trans = poly_features.fit_transform(X_train)
    
# Step 3: define and train a model
    model = LinearRegression()
    model.fit(x_train_trans, Y_train.ravel())
    
# Step 4: calculate bias and variance    
    y_train_predict = model.predict(x_train_trans)
#     y_val_predict = model.predict(x_trans)
    
    RMSE = np.sqrt(mean_squared_error(Y_train.ravel(),y_train_predict))
    R2 = r2_score(Y_train.ravel(),y_train_predict)
    MAE = mean_absolute_error(Y_train.ravel(),y_train_predict)
    train_rmse.append(RMSE)
    train_r2.append(R2)
    train_mae.append(MAE)
    
# Step 5: prediction
    x_new_min = X_train.min()
    x_new_max = X_train.max()

    X_NEW = np.linspace(x_new_min, x_new_max, 10).reshape(-1, 1) 

    X_NEW_TRANSF = poly_features.fit_transform(X_NEW)
    Y_NEW = model.predict(X_NEW_TRANSF)
           
    x10_label.append(X_NEW.tolist())
    y10_label.append(Y_NEW.tolist()) 
    
x10_label = np.array(x10_label)  
y10_label_poly = np.array(y10_label)  

Train_performance = pd.DataFrame({'R2':train_r2, 'RMSE':train_rmse,'MAE':train_mae})
Train_performance.mean()

R2        0.971166
RMSE    403.088100
MAE     324.040237
dtype: float64

In [17]:
y10_poly = pd.DataFrame(y10_label_poly)
y10_poly

df_all = df_del.reset_index()
df_all = df_all.drop(['index'], axis=1) # 컬럼을 제거 
df_all

df_new = pd.concat([y10_poly,df_all],axis=1)

df_new['result']= df_new['length'] == df_new['uni_length']

df_len_index = df_new[df_new['result']==False].index
df_len = df_new.drop(df_len_index)
df_len = df_len.reset_index()
df_len

y10 = df_len[[0,1,2,3,4,5,6,7,8,9]]
y10

seq_feature = df_len.iloc[:,26:10010]
seq_feature

A = df_len[['ndatapoints','temperature','ph']]
B = df_len[['mutation_type','unit']]

con1 = {'mutation_type' : {'Wild Type':0, 'Single':1,'Double':2,'Multiple':3, 'Modified':4,'Triple':5}}
B = B.replace(con1)

con2 = {'unit' : {'Hours':0, 'Minutes':1,'Days':2,'Seconds':3}}
B = B.replace(con2)

All_feature = pd.concat([A,B,seq_feature],axis=1)
All_feature

,ndatapoints,temperature,ph,mutation_type,unit,BIOPYTHON|Length,BIOPYTHON|Molecular_weight,BIOPYTHON|Aromaticity,BIOPYTHON|Instability_index,BIOPYTHON|Isoelectric_point,...,PROTSCALE|Coil,PROTSCALE|Nomalized_frequency_alpha_helix,PROTSCALE|Nomalized_frequency_beta_sheet,PROTSCALE|Nomalized_frequency_beta_turn,PROTSCALE|Total_beta_strand,PROTSCALE|Antiparallel_beta_strand,PROTSCALE|Parallel_beta_strand,PROTSCALE|AA_composition,PROTSCALE|AA_composition_UniProtKB,PROTSCALE|Relative_mutability
0,10,37,7.5,1,0,140,14459.9852,0.042857,25.462857,4.665708,...,135.410,146.15,137.10,139.75,134.32,131.63,143.55,868.7,877.15,11230
1,9,37,7.5,1,0,140,14459.9852,0.042857,25.462857,4.665708,...,135.410,146.15,137.10,139.75,134.32,131.63,143.55,868.7,877.15,11230
2,11,37,7.5,1,0,140,14459.9852,0.042857,25.462857,4.665708,...,135.410,146.15,137.10,139.75,134.32,131.63,143.55,868.7,877.15,11230
3,9,37,7.5,1,0,140,14459.9852,0.042857,25.462857,4.665708,...,135.410,146.15,137.10,139.75,134.32,131.63,143.55,868.7,877.15,11230
4,10,37,7.5,1,0,140,14459.9852,0.042857,25.462857,4.665708,...,135.410,146.15,137.10,139.75,134.32,131.63,143.55,868.7,877.15,11230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,3,0,7.2,0,1,89,9806.3142,0.056180,26.956180,9.831187,...,86.311,92.44,89.76,82.14,92.88,92.57,94.38,539.8,544.35,7008
128,7,0,7.2,0,1,89,9806.3142,0.056180,26.956180,9.831187,...,86.311,92.44,89.76,82.14,92.88,92.57,94.38,539.8,544.35,7008
129,12,23,4.8,1,0,98,11139.4540,0.102041,39.300000,6.962288,...,95.698,103.61,97.27,93.10,97.19,98.66,93.43,551.0,552.98,7765
130,12,23,4.8,1,0,98,11139.4540,0.102041,39.300000,6.962288,...,95.698,103.61,97.27,93.10,97.19,98.66,93.43,551.0,552.98,7765


In [18]:
from numpy import array, hstack, math
from numpy.random import uniform
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from tqdm import notebook
import time 
from sklearn.model_selection import KFold #회귀
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import ExtraTreesRegressor

Train_rmse_2=[]
Train_mae_2=[]
Train_R2_2=[]

Test_rmse_2=[]
Test_mae_2=[]
Test_R2_2=[]

index = []

X = All_feature
Y = y10
Y.columns = ['Y0','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9']
Y[['Y0']]

X = All_feature
Y = y10
n_iter = 0

kfold = KFold(n_splits = 5, shuffle=True, random_state=0)

for train_index, test_index in kfold.split(X,Y):
    xtrain, xtest = X.iloc[train_index], X.iloc[test_index]
    ytrain, ytest = Y.iloc[train_index], Y.iloc[test_index]
    
    est = ExtraTreesRegressor(random_state=0)
    model = MultiOutputRegressor(estimator=est)
    

    model.fit(xtrain, ytrain)
    ypred_train = model.predict(xtrain)

    ypred_test = model.predict(xtest)
    n_iter +=1

    train_size = xtrain.shape[0]
    val_size = xtest.shape[0]
    
    print('\n#{0} Train_set size: {1}, Validation_set size: {2}'
          .format(n_iter, train_size, val_size))

    index.append(test_index)
    
# 성능
    train_rmse2 = np.sqrt(mean_squared_error(ytrain, ypred_train))
    train_mae2 = mean_absolute_error(ytrain, ypred_train)
    train_R22= r2_score(ytrain, ypred_train)
    
    test_rmse2 = np.sqrt(mean_squared_error(ytest, ypred_test))
    test_mae2 = mean_absolute_error(ytest, ypred_test)
    test_R22= r2_score(ytest, ypred_test)

    Train_rmse_2.append(train_rmse2)
    Train_mae_2.append(train_mae2)
    Train_R2_2.append(train_R22)
    
    Test_rmse_2.append(test_rmse2)
    Test_mae_2.append(test_mae2)
    Test_R2_2.append(test_R22)
    
Y_MV = pd.DataFrame({'Train_rmse':Train_rmse_2,'Train_mae':Train_mae_2,'Train_R2':Train_R2_2,
                             'Test_rmse':Test_rmse_2, 'Test_mae':Test_mae_2,'Test_R2':Test_R2_2})
Y_MV.mean()


#1 Train_set size: 105, Validation_set size: 27

#2 Train_set size: 105, Validation_set size: 27

#3 Train_set size: 106, Validation_set size: 26

#4 Train_set size: 106, Validation_set size: 26

#5 Train_set size: 106, Validation_set size: 26


Train_rmse     22939.709318
Train_mae       4757.588981
Train_R2           0.973440
Test_rmse     117585.243235
Test_mae       27168.088486
Test_R2            0.497774
dtype: float64

In [19]:
import shap

class MultiOutputExplainer(shap.Explainer):
    """ Abstraction of Explainer for MultiOutputRegressor model
    """
    def __init__(self, model, *args, **kwargs):
        assert isinstance(model, MultiOutputRegressor)
        self.explainers = []
        self.expected_value = []
        for estimator in model.estimators_:
            explainer = shap.Explainer(estimator, *args, **kwargs)
            self.explainers.append(explainer)
            self.expected_value.append(explainer.expected_value)

    def shap_values(self, *args, **kwargs):
        shap_values = []
        for explainer in self.explainers:
            shap_values.append(explainer.shap_values(*args, **kwargs))
        return np.array(shap_values)

    def shap_interaction_values(self, *args, **kwargs):
        shap_interaction_values = []
        for explainer in self.explainers:
            shap_interaction_values.append(explainer.shap_interaction_values)
        return np.array(shap_interaction_values)
    
import shap
import numpy as np
from sklearn.multioutput import MultiOutputRegressor

ME = MultiOutputExplainer(model)
multi_shap = ME.shap_values(xtrain)
multi_shap

imp =[]
for i in range(0,10,1):
    est = model.estimators_[i]
    im = est.feature_importances_
    imp.append(im)
feature_importances = pd.DataFrame(imp)
T = feature_importances.T
T.columns = ['label 0','label 1','label 2','label 3','label 4','label 5','label 6','label 7','label 8','label 9']
T.index = All_feature.columns
mean = pd.DataFrame(T.mean(axis='columns'), columns =['Mean'])
feature_importance = pd.concat([T,mean],axis=1)
feature_importance

sort = feature_importance.sort_values(by = ['Mean'], ascending = False) # 내림차순 정렬
sort

Mean = sort[['Mean']]
Top10 = abs(Mean).sort_values('Mean', ascending = False).head(10)
Top10

s = Mean[abs(Mean['Mean']) <2.042836e-07]
s_index = s.index
SS = Mean.drop(s_index)
Feature_selection = All_feature.drop(columns = s.index.to_list())
Feature_selection

,ndatapoints,ph,mutation_type,unit,BIOPYTHON|Aromaticity,BIOPYTHON|Instability_index,PYBIOMED|AAComp|H_R,PYBIOMED|AAComp|K_R,PYBIOMED|AAComp|S_R,PYBIOMED|AAComp|W_R,...,PYBIOMED|Triad|646,PYBIOMED|Triad|741,PYBIOMED|Triad|744,PYBIOMED|Triad|762,PROTSCALE|E_C,PROTSCALE|N_C,PROTSCALE|Q_C,PROTSCALE|V_C,PROTSCALE|Hydrophobicity_scale_Fauchere,PROTSCALE|Hydrophobicity_indices_HPLC_75
0,10,7.5,1,0,0.042857,25.462857,4.286,0.714,2.857,2.857,...,1,0,0,0,18,3,6,19,23.87,-29.39
1,9,7.5,1,0,0.042857,25.462857,4.286,0.714,2.857,2.857,...,1,0,0,0,18,3,6,19,23.87,-29.39
2,11,7.5,1,0,0.042857,25.462857,4.286,0.714,2.857,2.857,...,1,0,0,0,18,3,6,19,23.87,-29.39
3,9,7.5,1,0,0.042857,25.462857,4.286,0.714,2.857,2.857,...,1,0,0,0,18,3,6,19,23.87,-29.39
4,10,7.5,1,0,0.042857,25.462857,4.286,0.714,2.857,2.857,...,1,0,0,0,18,3,6,19,23.87,-29.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,3,7.2,0,1,0.056180,26.956180,3.371,3.371,7.865,2.247,...,0,0,0,0,4,9,3,8,38.06,12.00
128,7,7.2,0,1,0.056180,26.956180,3.371,3.371,7.865,2.247,...,0,0,0,0,4,9,3,8,38.06,12.00
129,12,4.8,1,0,0.102041,39.300000,7.143,5.102,6.122,3.061,...,0,0,0,0,7,5,7,9,29.47,-10.19
130,12,4.8,1,0,0.102041,39.300000,7.143,5.102,6.122,3.061,...,0,0,0,0,7,5,7,9,29.47,-10.19


In [20]:
# Extratree
from tqdm import notebook
import time 
from sklearn.model_selection import KFold #회귀
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

Train_rmse_2=[]
Train_mae_2=[]
Train_R2_2=[]

Test_rmse_2=[]
Test_mae_2=[]
Test_R2_2=[]

index = []

X = Feature_selection
Y = y10
n_iter = 0

kfold = KFold(n_splits = 5, shuffle=True, random_state=0)

for train_index, test_index in kfold.split(X,Y):
    xtrain, xtest = X.iloc[train_index], X.iloc[test_index]
    ytrain, ytest = Y.iloc[train_index], Y.iloc[test_index]
    
    est = ExtraTreesRegressor(random_state=0)
    model = MultiOutputRegressor(estimator=est)

    model.fit(xtrain, ytrain)
    ypred_train = model.predict(xtrain)

    ypred_test = model.predict(xtest)
    n_iter +=1
    

    train_size = xtrain.shape[0]
    val_size = xtest.shape[0]
    
    print('\n#{0} Train_set size: {1}, Validation_set size: {2}'
          .format(n_iter, train_size, val_size))

    index.append(test_index)
    
    train_rmse2 = np.sqrt(mean_squared_error(ytrain, ypred_train))
    train_mae2 = mean_absolute_error(ytrain, ypred_train)
    train_R22= r2_score(ytrain, ypred_train)
    
    test_rmse2 = np.sqrt(mean_squared_error(ytest, ypred_test))
    test_mae2 = mean_absolute_error(ytest, ypred_test)
    test_R22= r2_score(ytest, ypred_test)

    Train_rmse_2.append(train_rmse2)
    Train_mae_2.append(train_mae2)
    Train_R2_2.append(train_R22)
    
    Test_rmse_2.append(test_rmse2)
    Test_mae_2.append(test_mae2)
    Test_R2_2.append(test_R22)
    
Y_MV = pd.DataFrame({'Train_rmse':Train_rmse_2,'Train_mae':Train_mae_2,'Train_R2':Train_R2_2,
                             'Test_rmse':Test_rmse_2, 'Test_mae':Test_mae_2,'Test_R2':Test_R2_2})
Y_MV.mean()


#1 Train_set size: 105, Validation_set size: 27

#2 Train_set size: 105, Validation_set size: 27

#3 Train_set size: 106, Validation_set size: 26

#4 Train_set size: 106, Validation_set size: 26

#5 Train_set size: 106, Validation_set size: 26


Train_rmse     22939.709318
Train_mae       4757.588978
Train_R2           0.973440
Test_rmse     117031.111574
Test_mae       27243.460013
Test_R2            0.497852
dtype: float64

In [21]:
# Extratree
from tqdm import notebook
import time 
from sklearn.model_selection import KFold #회귀
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

Train_rmse_2=[]
Train_mae_2=[]
Train_R2_2=[]

Test_rmse_2=[]
Test_mae_2=[]
Test_R2_2=[]

index = []
pred_test =[]
X = Feature_selection
Y = y10
n_iter = 0

kfold = KFold(n_splits = 5, shuffle=True, random_state=0)

for train_index, test_index in kfold.split(X,Y):
    xtrain, xtest = X.iloc[train_index], X.iloc[test_index]
    ytrain, ytest = Y.iloc[train_index], Y.iloc[test_index]
    
    est = ExtraTreesRegressor(random_state=0)
    model = MultiOutputRegressor(estimator=est)
    
    model.fit(xtrain, ytrain)
    ypred_train = model.predict(xtrain)

    ypred_test = model.predict(xtest)
    n_iter +=1
    
    train_size = xtrain.shape[0]
    val_size = xtest.shape[0]
    
    print('\n#{0} Train_set size : {1}, Validation_set size: {2}'
          .format(n_iter, train_size, val_size))
    print('{0} Validation_set index:{1}'.format(n_iter,test_index))
    pred_test.append(ypred_test)
    index.append(test_index)

    train_rmse2 = np.sqrt(mean_squared_error(ytrain, ypred_train))
    train_mae2 = mean_absolute_error(ytrain, ypred_train)
    train_R22= r2_score(ytrain, ypred_train)
    
    test_rmse2 = np.sqrt(mean_squared_error(ytest, ypred_test))
    test_mae2 = mean_absolute_error(ytest, ypred_test)
    test_R22= r2_score(ytest, ypred_test)

    Train_rmse_2.append(train_rmse2)
    Train_mae_2.append(train_mae2)
    Train_R2_2.append(train_R22)
    
    Test_rmse_2.append(test_rmse2)
    Test_mae_2.append(test_mae2)
    Test_R2_2.append(test_R22)
    
Y_MV = pd.DataFrame({'Train_rmse':Train_rmse_2,'Train_mae':Train_mae_2,'Train_R2':Train_R2_2,
                             'Test_rmse':Test_rmse_2, 'Test_mae':Test_mae_2,'Test_R2':Test_R2_2})
Y_MV.mean()


#1 Train_set size : 105, Validation_set size: 27
1 Validation_set index:[  2   7   8  10  16  22  24  26  30  43  44  45  56  61  66  68  78  91
  93  95  96  97  99 100 101 108 109]

#2 Train_set size : 105, Validation_set size: 27
2 Validation_set index:[  3   6  13  15  33  40  48  50  51  52  54  59  60  62  63  71  73  84
  86 104 110 115 118 121 125 126 129]

#3 Train_set size : 106, Validation_set size: 26
3 Validation_set index:[  0   1   4   5  11  17  18  27  28  34  38  41  42  53  76  85  89  92
  98 106 107 112 113 123 128 130]

#4 Train_set size : 106, Validation_set size: 26
4 Validation_set index:[ 14  19  20  23  29  31  32  35  49  55  57  64  69  74  75  79  80  82
  90  94 105 111 116 124 127 131]

#5 Train_set size : 106, Validation_set size: 26
5 Validation_set index:[  9  12  21  25  36  37  39  46  47  58  65  67  70  72  77  81  83  87
  88 102 103 114 117 119 120 122]


Train_rmse     22939.709318
Train_mae       4757.588978
Train_R2           0.973440
Test_rmse     117031.111574
Test_mae       27243.460013
Test_R2            0.497852
dtype: float64

In [30]:
val_best_index = pd.DataFrame(index).T[[3]]
val_best_index.columns=['3 model index']
val_best_index

raw_index = y10.reset_index()
P_merge = pd.merge(raw_index,val_best_index, left_on ='index',right_on = '3 model index', how = 'right')
P_merge

# raw value
raw_best = P_merge.drop(['index','3 model index'],axis=1).dropna()
raw_best

from functools import reduce
import operator
p = pd.DataFrame(pred_test).T[[3]][3]
arr= p.array
pred_best = pd.DataFrame(list(reduce(operator.add, arr)))
pred_best

Train_rmse_2=[]
Train_mae_2=[]
Train_R2_2=[]

Test_rmse_2=[]
Test_mae_2=[]
Test_R2_2=[]

y = raw_best
yp = pred_best

for i in range(y.shape[0]):
    test_rmse2 = np.sqrt(mean_squared_error(y.iloc[i], yp.iloc[i]))
    test_mae2 = mean_absolute_error(y.iloc[i], yp.iloc[i])
    test_R22= r2_score(y.iloc[i], yp.iloc[i])
    
    Test_rmse_2.append(test_rmse2)
    Test_mae_2.append(test_mae2)
    Test_R2_2.append(test_R22)
    
Y_MV = pd.DataFrame({'Test_rmse':Test_rmse_2, 'Test_mae':Test_mae_2,'Test_R2':Test_R2_2})
Y_MV.mean()

index_perform = pd.concat([P_merge[['index']], Y_MV],axis=1)
index_perform

uni = df_len[['uniprot_id','entry']].reset_index()
final = pd.merge(uni,index_perform, left_on ='index',right_on = 'index', how = 'right')
final

final.sort_values('Test_R2',ascending=False)

,index,uniprot_id,entry,Test_rmse,Test_mae,Test_R2
11,64.0,P37840,T-0129,0.000003,0.000002,1.000000e+00
10,57.0,P37840,T-0122,0.030287,0.023325,9.931105e-01
23,124.0,P10997,T-1103,1.221145,0.859479,9.606004e-01
18,90.0,P47929,T-0664,0.426202,0.352346,9.254486e-01
19,94.0,P47929,T-0668,0.793723,0.532238,9.059194e-01
21,111.0,P01594,T-0993,154014.314939,128892.059403,8.839350e-01
7,35.0,P37840,T-0100,0.115595,0.086744,8.717310e-01
0,14.0,P37840,T-0079,0.125781,0.078839,8.588059e-01
3,23.0,P37840,T-0088,0.165551,0.121460,7.887569e-01
9,55.0,P37840,T-0120,0.141046,0.111624,7.724682e-01


In [31]:
vi_mer = pd.merge(final,df, left_on ='entry',right_on = 'entry', how = 'left')
vi_mer

y.columns=['poly0','poly1','poly2','poly3','poly4','poly5','poly6','poly7','poly8','poly9']
y

yp.columns=['pred0','pred1','pred2','pred3','pred4','pred5','pred6','pred7','pred8','pred9']
yp

con = pd.concat([vi_mer, y, yp],axis=1)
con

visual = con.sort_values(by = ['Test_R2'], ascending = False)
visual

y_poly = visual.iloc[:,10005:10015]
y_poly

y_pred = visual.iloc[:,10015:10025]
y_pred

,pred0,pred1,pred2,pred3,pred4,pred5,pred6,pred7,pred8,pred9
11,-2.892452e-02,0.139338,0.193170,0.270586,0.412197,0.595238,7.675941e-01,8.818309e-01,0.929220,9.737666e-01
10,-2.668356e-01,0.585348,0.981814,1.084648,1.039171,0.961072,9.235471e-01,9.444362e-01,0.973362,8.788658e-01
23,5.749944e-02,0.118460,0.137221,0.389186,2.028423,6.034313,1.215819e+01,1.787000e+01,17.304908,2.100000e-01
18,1.017639e-13,0.720462,0.820557,-0.374717,-2.056989,-2.838608,-1.656738e+00,1.322550e+00,3.800185,4.400000e-01
19,1.257143e-01,0.925980,1.464193,0.885112,-0.138438,-0.340395,1.203702e+00,4.141881e+00,5.910904,8.014286e-01
21,1.981085e+05,76742.361978,227951.324631,500892.312016,774336.322198,965108.646689,1.036529e+06,1.006852e+06,957707.553513,1.042540e+06
7,-6.203836e-04,0.283573,0.422884,0.516575,0.653572,0.775842,8.838801e-01,9.428990e-01,0.958703,1.017780e+00
0,3.692857e-02,0.628575,0.817391,0.861778,0.887362,0.928253,9.683023e-01,9.823611e-01,0.977541,1.034469e+00
3,2.829934e-01,0.975338,0.988673,0.871951,0.843128,0.904809,9.598937e-01,9.272217e-01,0.857218,1.047536e+00
9,-2.668356e-01,0.585348,0.981814,1.084648,1.039171,0.961072,9.235471e-01,9.444362e-01,0.973362,8.788658e-01
